# Typical 1st & 2nd Order IIR-Filters in Audio Signal Processing

*This jupyter notebook is part of a [collection of notebooks](../index.ipynb) on various topics of Digital Signal Processing. Please direct questions and suggestions to [Sascha.Spors@uni-rostock.de](mailto:Sascha.Spors@uni-rostock.de).*

## Preliminaries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import audiofilter as af
np.set_printoptions(precision=16)  # print filter coeff in very high precision
fs = 48000  # sampling frequency in Hz

We use standard LaTex-style citing within the text. At the moment there is no commonly accepted long-term solution for rendering a reference list in a Jupyter notebook automatically, so we leave the bibtex information here as plain text as well. However, it's working with enabled `LaTeX environments for Jupyter` of the `Nbextensions` using the external `biblio.bib` file. Otherwise, digital sources appear linked with their URLs.

## Introduction

We recollect the most common analog transfer functions (Laplace domain) of 1st and 2nd order IIR audio filters and derive bilinear transforms (transfer functions in z-domain), applying prewarping for cut/mid frequency and for quality/bandwidth.

If you are interested in a Matlab based implementation of these filters, please have a look at the TU Berlin project [AKTools](https://www.ak.tu-berlin.de/menue/publications/open_research_tools/aktools/), cf. Fabian Brinkmann, Stefan Weinzierl: "AKtools -- An Open Software Toolbox for Signal Acquisition, Processing, and Inspection in Acoustics." In: *142nd AES Convention, Berlin, Germany, 2017*, e-Brief 309.

Although more sophisticated filter designs exist, the discussed filter characteristics solve many IIR based audio filtering tasks, such as in a typical digital audio workstation (DAW) and available [plugins](https://plugins.iem.at/), in digital mixing consoles, loudspeaker management systems, power amplifiers with DSP capabilities and audio player software.

Higher order IIR filters can be built from second order filters. A series connection of the discussed filter types achieves more complex transfer functions often being used in the mentioned applications. A so called graphical equalizer is a prominent example for that, which is however not restricted to build from series connection.

### Standardized vs. Non-Standardized Filter Characteristics
For the **standardized** lowpass, highpass, bandpass, bandstop and allpass filters, the Laplace transfer functions and their coefficients are well known from literature e.g. \cite{Moschytz1981}, [\cite{TietzeSchenk2002}](http://www.tietze-schenk.de/).

For shelving and parametric filters **no standardization w.r.t. the pole/zero qualities and cut-frequencies and thus positions of poles and zeros** exists. Based on \cite{Zoelzer2002,Zoelzer2005},[\cite{Zoelzer2008}](https://link.springer.com/chapter/10.1007/978-3-540-34301-1_15),[\cite{Zoelzer2011}](https://doi.org/10.1002/9781119991298) (in which these filters are derived with the help of allpass filters) and some other articles \cite{Kimball1938}, \cite{Bristow-Johnson}, [\cite{Bristow-Johnson1994}](http://www.aes.org/e-lib/browse.cfm?elib=6326), [\cite{Orfanidis1997}](http://www.aes.org/e-lib/browse.cfm?elib=7854), [\cite{Christensen2003}](http://www.aes.org/e-lib/browse.cfm?elib=12429) widely used approaches can be found.

\cite{Christensen2003} discusses the generalized form of the biquad transfer function for progammable filters.

Three characteristics (labeled as types **I, II and III**) are most prominent, leading to different transfer functions for the same parameter set. This regularly leads to misconceptions. Recently, the Audio Engineering Society (AES) started to readdress this [issue](http://www.aes.org/events/141/workshops/?ID=5197) at 141st AES Convention in Los Angeles, 2016.

### Overview of Filter Transfer Functions

The follwing filter characteristics are discussed in this notebook:

[1st order lowpass](#lowpass1st)

[2nd order lowpass](#lowpass2nd)

[1st order highpass](#highpass1st) 

[2nd order highpass](#highpass2nd)

[2nd order bandpass](#bandpass2nd) 

[2nd order bandstop](#bandstop2nd)

[1st order allpass](#allpass1st) 

[2nd order allpass](#allpass2nd)

[2nd order PEQ](#peq2nd) type **I,I,III**

[1st order low-shelving](#lowshv1st) type **I,I,III**

[2nd order low-shelving](#lowshv2nd) type **I,I,III**

[1st order high-shelving](#highshv1st) type **I,I,III**

[2nd order low-shelving](#highshv2nd) type **I,I,III**

## Bilinear Transform of an IIR Biquad

The transformation of the analog filter prototypes (given as Laplace transfer function) towards the digital prototype (given as z-domain transfer function) is performed with the [bilinear transform](bilinear_transform.ipynb) \cite[Ch. 7.1.2]{OppenheimSchaferBuck2004}) applying frequency and bandwidth prewarping techniques.

There are other approaches such as the impulse invariant method, the matched z-transform method and higher order Taylor series truncations \cite{Clark2000}, \cite{Gunness2007}, \cite{Alaoui2007}, \cite{Schmidt2010}, \cite{Alaoui2010} that yield higher precision matching of the analog and digital transform. For didactical purpose we restrict ourselves to the bilinear transform of IIR biquads.

Let us define the angular frequency $\omega$ in rad/s, $\mathrm{i}^2=-1$ and the sampling frequency $f_s$ in Hz. The connection between the Laplace domain and the z-domain is given by

\begin{equation}
s = \mathrm{i}\,\omega \qquad z = \mathrm{e}^{\frac{s}{f_s}} \qquad \stackrel{\text{linearization from Taylor series}}{\rightarrow} \qquad s = 2\,f_s\,\frac{z-1}{z+1}.
\end{equation}

The continuous and discrete time transfer functions $H(s)$ and $H(z)$, respectively for the 2nd order filter (biquad) are given as 

\begin{equation}
H(s) = \frac{B_0\,s^2+B_1\,s+B_2}{A_0\,s^2+A_1\,s+A_2}
\qquad
H(z) = \frac{b_0+b_1\,z^{-1}+b_2\,z^{-2}}{1+a_1\,z^{-1}+a_2\,z^{-2}}
\end{equation}

following typical conventions for the denotation of the coefficients (Python and Matlab compatible).

The bilinear transform rule $s = 2\,f_s\,\frac{z-1}{z+1}$ can be inserted to $H(s)$ and the result can be brought to form of biquad given as $H(z)$ above. Then the coefficients for the biquad are generally given by, cf. [bilinear transform](bilinear_transform.ipynb)

\begin{align}
b_0 &= \frac{B_2+2\,B_1\,f_s+4\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
b_1 &= \frac{2\,B_2-8\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
b_2 &= \frac{B_2-2\,B_1\,f_s+4\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
a_1 &= \frac{2\,A_2-8\,A_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
a_2 &= \frac{A_2-2\,A_1\,f_s+4\,A_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}.
\end{align}

The function `bilinear_biquad()` realizes the bilinear transformation based on the above given equations.

## Frequency and Bandwidth Prewarping

A so called prewarping \cite[eq. (7.28)]{OppenheimSchaferBuck2004} of the cut/mid frequency $f_c$/$f_m$ in Hz

\begin{equation}
\omega_{c} \Leftarrow 2\,f_s\,\tan{\left(\pi\frac{f_{c}}{f_s}\right)}
\qquad
\omega_{m} \Leftarrow 2\,f_s\,\tan{\left(\pi\frac{f_{m}}{f_s}\right)}
\end{equation}

is realized in the function `prewarping_f()`.

The connection between the bandwidth $BW_\text{oct}$ in octaves and the corresponding bandpass-related quality $Q_\text{BP}$ is given as

\begin{equation}
BW_\text{oct} = \frac{2}{\log_\mathrm{e}(2)}\,\sinh^{-1}(\frac{1}{2\,Q_\text{BP}})
\qquad
Q_\text{BP} = \frac{1}{2\,\sinh(\frac{\log_\mathrm{e}(2)}{2}\,BW_\text{oct})}
\end{equation}

with the natural logarithm $\log_\mathrm{e}(\cdot)$ and the hyperbolic sine $\sinh(\cdot)$. The functions `bw_from_q()` and `q_from_bw()` do this job.

The prewarping of the bandpass quality for the parametric, bandpass and bandstop filters can be done with

* **tan**-prewarping, cf. [\cite{Clark2000}](http://www.aes.org/e-lib/browse.cfm?elib=12069)

\begin{equation}
Q_\text{BP} \Leftarrow \frac{\frac{\pi\,f_m}{f_s}}{\tan\left(\frac{\pi\,f_m}{f_s}\right)} Q_\text{BP}
\end{equation}

* **cos**-prewarping, cf. \cite{Thaden1997}

\begin{equation}
Q_\text{BP} \Leftarrow \cos(\pi\,\frac{f_m}{f_s}) Q_\text{BP}
\end{equation}

* **sin**-prewarping of the bandwidth in octaves, cf. \cite{Bristow-Johnson1994}

\begin{equation}
BW_\text{oct} \Leftarrow \frac{2\,\pi\,\frac{f_m}{f_s}}{\sin(2\,\pi\,\frac{f_m}{f_s})} BW_\text{oct}
\end{equation}

exhibiting slightly different characteristics. The function `prewarping_q()` handles the discussed cases.

## Plotting Routine
We establish a plotting routine for the `bode_plot()`, i.e. the magnitude and phase spectrum of the filter's transfer function. For that we use the `freqs()` and `freqz()` functions implemented in the `signal` package. We also use the self-implemented `zplane()` function for plotting zeros and poles in the z-plane.

## Example

### Without Frequency Prewarping
A first example demonstrates the application of the bilinear transform and plotting routine. We use a lowpass filter (simplest RC circuit) with the cut frequency $f_c$ in Hz, which is linked to $\omega_c=2\,\pi\,f_c=\frac{1}{R\,C}$. For the discrete time filter we need to specify the sampling frequency $f_s$ in Hz.

In [ ]:
# example lowpass 1st order without frequency pre-warping
fc = 10000  # -3dB cut frequency in Hz
wg = 2*np.pi*fc  # rad/s

B = np.array([0., 0., 1.])  # Laplace transfer function
A = np.array([0., 1./wg, 1.])

b, a = af.bilinear_biquad(B, A, fs)  # z-transfer function
bsig, asig = sig.bilinear(B, A, fs)

# check if equivalent
np.testing.assert_allclose(b, bsig)
np.testing.assert_allclose(a, asig)

af.bode_plot(None, B, A, b, a, fs)

### With Frequency Prewarping

Now, do the same with frequency prewarping, in order that the intended 'analog' $f_c$ matches the 'digital' $f_c$ in discrete time domain.

In [ ]:
# example lowpass 1st order with frequency pre-warping
fc = 10000  # -3dB cut frequency in Hz
wc = 2*np.pi*fc  # rad/s
wp = af.prewarping_f(fc, fs)

B = np.array([0., 0., 1.])
A = np.array([0., 1./wc, 1.])
Bp = np.array([0., 0., 1.])
Ap = np.array([0., 1./wp, 1.])
b, a = af.bilinear_biquad(Bp, Ap, fs) 

af.bode_plot(None, B, A, b, a, fs)

## Definition of 1st / 2nd Order Filter Transfer Functions

In the [audiofilter.py](audiofilter.py) we implement functions to calculate filter coefficients of lowpass, highpass, allpass, bandpass, bandstop, shelving and parametric filters. These characteristics are very often used in audio applications, cf. e.g. [\cite{TietzeSchenk2002}](http://www.tietze-schenk.de/), \cite{Moschytz1981}, \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}, \cite{Bristow-Johnson1994}.

<a id="lowpass1st"></a>
## 1st Order Lowpass `lp1st`

\cite{Tietze1978}, \cite{Moschytz1981}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
0& 0& 1& 0& \frac{1}{\omega_\text{c}}&1\\ 
\hline   	 		
\end{array}

with cut frequency $\omega_c=2\pi f_c$ in rad/s

In [ ]:
fc = 1000  # Hz
B, A, b, a = af.biquad_lp1st(fc, fs)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="lowpass2nd"></a>
## 2nd Order Lowpass `lp2nd`

\cite{Tietze1978}, \cite{Moschytz1981}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
0& 0& 1& \frac{\text{b}_\text{i}}{\omega_\text{c}^2}& \frac{\text{a}_\text{i}}{\omega_\text{c}}&1\\ 
\hline   	 		
\end{array}

with cut frequency $\omega_c=2 \pi f_c$ in rad/s and filter characteristic coefficients $\text{b}_\text{i}$ and $\text{a}_\text{i}$

In [ ]:
fc = 1000  # Hz
bi = 1
ai = np.sqrt(2)  # Butterworth filter type
B, A, b, a = af.biquad_lp2nd(fc, fs, bi, ai)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highpass1st"></a>
## 1st Order Highpass `hp1st`

\cite{Tietze1978}, \cite{Moschytz1981}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
0& \frac{1}{\omega_\text{c}}& 0& 0& \frac{1}{\omega_\text{c}}& 1\\ 
\hline   	 		
\end{array}

with cut frequency $\omega_c=2 \pi f_c$ in rad/s

In [ ]:
fc = 1000  # Hz
B, A, b, a = af.biquad_hp1st(fc, fs)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highpass2nd"></a>
## 2nd Order Highpass `hp2nd`

\cite{Tietze1978}, \cite{Moschytz1981}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
\frac{1}{\omega_\text{c}^2}& 0& 0& \frac{1}{\omega_\text{c}^2}& \frac{\text{a}_\text{i}}{\omega_\text{c}}& \text{b}_\text{i}\\ 
\hline   	 		
\end{array}

with cut frequency $\omega_c$ in rad/s and filter characteristic coefficients $\text{b}_\text{i}$ and $\text{a}_\text{i}$

In [ ]:
fc = 1000  # Hz
bi = 1  # filter characteristic coeff
ai = np.sqrt(2)  # here for Butterworth
B, A, b, a = af.biquad_hp2nd(fc, fs, bi, ai)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="bandpass2nd"></a>
## 2nd Order Bandpass `bp2nd`

\cite{Tietze1978}, \cite{Moschytz1981}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
0& \frac{1}{Q_\text{BP}\,\omega_\text{m}}& 0& \frac{1}{\omega_\text{m}^2}& \frac{1}{Q_\text{BP}\,\omega_\text{m}}&1\\ 
\hline   	 		
\end{array}

with mid frequency $\omega_m=2 \pi f_m$ in rad/s and quality $Q_\text{BP}$

In [ ]:
BW = 2  # bandwidth in octaves
fm = 1000  # Hz

flow =  2**(-BW/2) * fm  # lower cut  (-3 dB) frequency in Hz 
fhigh = 2**(+BW/2) * fm  # higher cut (-3 dB) frequency in Hz 
QBP = af.q_from_bw(BW)
np.testing.assert_allclose(QBP, fm / (fhigh-flow))
QWarpType = "cos" # sin, cos, tan
B, A, b, a = af.biquad_bp2nd(fm, QBP, fs, QWarpType)
af.bode_plot(None, B, A, b, a, fs)

print("flow=", flow, "Hz")
print("fmid=", fm, "Hz")
print("fhigh=", fhigh, "Hz")
print("QBP=", QBP)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="bandstop2nd"></a>
## 2nd Order Bandstop `bs2nd`

\cite{Tietze1978}, \cite{Moschytz1981}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
\frac{1}{\omega_\text{m}^2}& 0& 1& \frac{1}{\omega_\text{m}^2}& \frac{1}{Q_\text{BS}\,\omega_\text{m}}&1\\ 
\hline   	 		
\end{array}

with mid frequency $\omega_m=2 \pi f_m$ in rad/s and quality $Q_\text{BS}$

In [ ]:
fm = 1000  # Hz
QBP = 2/3
QWarpType = "cos"
B, A, b, a = af.biquad_bs2nd(fm, QBP, fs, QWarpType)
af.bode_plot(None, B, A, b, a, fs)

print("flow=", flow, "Hz")
print("fmid=", fm, "Hz")
print("fhigh=", fhigh, "Hz")
print("QBP=", QBP)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="allpass1st"></a>
## 1st Order Allpass `ap1st`

\cite{Tietze1978}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
0 & \frac{-\text{a}_\text{i}}{\omega_c} & 1 &0 & \frac{+\text{a}_\text{i}}{\omega_c} & 1\\ 
\hline   	 		
\end{array}

with cut frequency $\omega_c=2 \pi f_c$ in rad/s and filter characteristic coefficient $\text{a}_\text{i}$

In [ ]:
fc = 1000  # Hz
ai = 1
B, A, b, a = af.biquad_ap1st(fc, ai, fs)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="allpass2nd"></a>
## 2nd Order Allpass `ap2nd`

\cite{Tietze1978}

\begin{array}{|c|c|c|c|c|c|}
\hline
B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
\frac{\text{b}_\text{i}}{\omega_c^2} & \frac{-\text{a}_\text{i}}{\omega_c} & 1 & \frac{\text{b}_\text{i}}{\omega_c^2} & \frac{+\text{a}_\text{i}}{\omega_c} & 1\\ 
\hline   	 		
\end{array}

with cut frequency $\omega_c=2 \pi f_c$ in rad/s and filter characteristic coefficients $\text{b}_\text{i}$ and $\text{a}_\text{i}$

In [ ]:
fc = 1000  #  Hz
bi = 1
ai = np.sqrt(2)
B, A, b, a = af.biquad_ap2nd(fc, bi, ai, fs)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="peq2nd"></a>
## 2nd Order PEQ `peq2nd`

cf. \cite{Bristow-Johnson1994}, \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}

\begin{array}{|l|c|c|c|c|c|c|}
\hline
& B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
G\leq0 & \frac{1}{\omega_\text{m}^2} & 
\frac{\gamma}{Q_\text{BP}\,\omega_\text{m}} & 1 & \frac{1}{\omega_\text{m}^2} & \frac{\gamma/g}{Q_\text{BP}\,\omega_\text{m}} & 1\\ 
\hline   	
G>0 & \frac{1}{\omega_\text{m}^2} & \frac{\delta}{Q_\text{BP}\,\omega_\text{m}} & 1 & \frac{1}{\omega_\text{m}^2} & \frac{\delta/g}{Q_\text{BP}\,\omega_\text{m}} & 1\\
\hline
\end{array}        

with  $g=10^{\frac{G}{20}}$, mid frequency $\omega_m=2 \pi f_m$ in rad/s, quality $Q_\text{BP}$ and gain $G$ in dB
        
* **type I:** 
$\gamma=g\,,\delta=g$ 

for $G\gg 0$ bandwidth is similar to that of the 2nd order bandpass

for $G\ll 0$ bandwidth is similar to that of the 2nd order bandstop

* **type II:**
$\gamma=1\,,\delta=g$

for $G\gg 0$ bandwidth is similar to that of the 2nd order bandpass

for $G<0$ the transfer function is inverted from $G>0$

* **type III (one-half pad loss):**
$\gamma=g^{1/2}\,,\delta=g^{1/2}$

cut frequencies are located at $G/2$



In [ ]:
BW = 2  # bandwidth in octaves
fm = 1000  # Hz
G = 12  # dB
Q = af.q_from_bw(BW)
PEQType = "III"
QWarpType = "cos"
B, A, b, a = af.biquad_peq2nd(fm, G, Q, fs, PEQType, QWarpType)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="lowshv1st"></a>
## 1st Order Low-Shelving `lshv1st`

cf. \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}

\begin{array}{|l|c|c|c|c|c|c|}
\hline
& B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
G\leq0 & 0& \frac{1}{\omega_\text{c}} & \alpha^{+2} & 0& \frac{1}{\omega_\text{c}}& g^{-1}\, \alpha^{+2}\\ 
\hline   	
G>0 & 0& \frac{1}{\omega_\text{c}}& g^{+1}\, \alpha^{-2} & 0& \frac{1}{\omega_\text{c}}& \alpha^{-2}\\
\hline
\end{array}

with $g=10^{\frac{G}{20}}$, cut frequency $\omega_c=2 \pi f_c$ in rad/s and gain $G$ in dB

* **type I:**
$\alpha=1$ 

for $G\gg 0$ cut frequency is at $G$-3 dB

for $G\ll 0$ cut frequency is at $G$+3 dB

* **type II:**
$\alpha=g^{+1/2}$

for $G\gg 0$ cut frequency is at +3 dB

for $G\ll 0$ cut frequency is at -3 dB
		
* **type III (one-half pad loss):**
$\alpha=g^{+1/4}$

cut frequency is located at $G/2$

In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
B, A, b, a = af.biquad_lshv1st(fc, G, fs, ShvType)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="lowshv2nd"></a>
## 2nd Order Low-Shelving `lshv2nd`

cf. \cite{Bristow-Johnson1994}, \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}

\begin{array}{|l|c|c|c|c|c|c|}
\hline
& B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
G\leq0 &
\frac{1}{\omega_\text{c}^2}& 
\frac{\alpha^{+1}}{Q_\text{z}\,\omega_\text{c}}& 
\alpha^{+2}& 
\frac{1}{\omega_\text{c}^2}& 
\frac{g^{-1/2}\,\alpha^{+1}}{Q_\text{p}\,\omega_\text{c}}& 
g^{-1}\,\alpha^{+2}\\ 
\hline   	
G>0 &
\frac{1}{\omega_\text{c}^2} &
\frac{g^{+1/2}\,\alpha^{-1}}{Q_\text{z}\,\omega_\text{c}}& 
g^{+1}\,\alpha^{-2}& 
\frac{1}{\omega_\text{c}^2} & 
\frac{\alpha^{-1}}{Q_\text{p}\,\omega_\text{c}}& 
\alpha^{-2}\\
\hline
\end{array}

with $g=10^{\frac{G}{20}}$, cut frequency $\omega_c=2 \pi f_c$ in rad/s, gain $G$ in dB, zero's quality $Q_z$ and pole's quality $Q_p$

* **type I:**
$\alpha=1$ 

for $G\gg 0$ cut frequency is at $G$-3 dB if $Q_p=Q_z=1/\sqrt{2}$

for $G\ll 0$ cut frequency is at $G$+3 dB if $Q_p=Q_z=1/\sqrt{2}$

* **type II:**
$\alpha=g^{+1/2}$

for $G\gg 0$ cut frequency is at +3 dB if $Q_p=Q_z=1/\sqrt{2}$

for $G\ll 0$ cut frequency is at -3 dB if $Q_p=Q_z=1/\sqrt{2}$
		
* **type III (one-half pad loss):**
$\alpha=g^{+1/4}$

cut frequency is located at $G/2$

In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
Qz = 1/np.sqrt(2)  # Butterworth quality
Qp = Qz
B, A, b, a = af.biquad_lshv2nd(fc, G, Qz, Qp, fs, ShvType)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highshv1st"></a>
## 1st Order High-Shelving `hshv1st`

cf. \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}

\begin{array}{|l|c|c|c|c|c|c|}
\hline
& B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
G\leq0 & 0& \frac{\alpha^{+2}}{\omega_\text{c}}& 1& 0& \frac{g^{-1}\,\alpha^{+2}}{\omega_\text{c}}& 1\\ 
\hline   	
G>0 & 0& \frac{g^{+1} \,\alpha^{-2}}{\omega_\text{c}}& 1& 0& \frac{\alpha^{-2}}{\omega_\text{c}}& 1\\
\hline
\end{array}        

with $g=10^{\frac{G}{20}}$, cut frequency $\omega_c=2 \pi f_c$ in rad/s and gain $G$ in dB

* **type I:**
$\alpha=1$ 

for $G\gg 0$ cut frequency is at $G$-3 dB

for $G\ll 0$ cut frequency is at $G$+3 dB

* **type II:**
$\alpha=g^{+1/2}$

for $G\gg 0$ cut frequency is at +3 dB

for $G\ll 0$ cut frequency is at -3 dB
		
* **type III (one-half pad loss):**
$\alpha=g^{+1/4}$

cut frequency is located at $G/2$



In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
B, A, b, a = af.biquad_hshv1st(fc, G, fs, ShvType)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highshv2nd"></a>
## 2nd Order High-Shelving `hshv2nd`

cf. \cite{Bristow-Johnson1994}, \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}

\begin{array}{|l|c|c|c|c|c|c|}
\hline
& B_0 & B_1 & B_2 & A_0 & A_1 & A_2\\
\hline
G\leq0&
\frac{\alpha^{+2}}{\omega_\text{c}^2} & 
\frac{\alpha^{+1}}{Q_\text{z}\,\omega_\text{c}} & 
1 & 
\frac{g^{-1}\,\alpha^{+2}}{\omega_\text{c}^2} & 
\frac{g^{-1/2}\,\alpha^{+1}}{Q_\text{p}\,\omega_\text{c}}      & 1\\ 
\hline   	
G>0 &
\frac{g^{+1}\,\alpha^{-2}}{\omega_\text{c}^2}& 
\frac{g^{+1/2}\,\alpha^{-1}}{Q_\text{z}\,\omega_\text{c}}& 
1& 
\frac{\alpha^{-2}}{\omega_\text{c}^2}& 
\frac{\alpha^{-1}}{Q_\text{p}\,\omega_\text{c}}& 
1\\
\hline
\end{array}

with $g=10^{\frac{G}{20}}$, cut frequency $\omega_c=2 \pi f_c$ in rad/s, gain $G$ in dB, zero's quality $Q_z$ and 
pole's quality $Q_p$

* **type I:**
$\alpha=1$ 

for $G\gg 0$ cut frequency is at $G$-3 dB if $Q_p=Q_z=1/\sqrt{2}$

for $G\ll 0$ cut frequency is at $G$+3 dB if $Q_p=Q_z=1/\sqrt{2}$

* **type II:**
$\alpha=g^{+1/2}$

for $G\gg 0$ cut frequency is at +3 dB if $Q_p=Q_z=1/\sqrt{2}$

for $G\ll 0$ cut frequency is at -3 dB if $Q_p=Q_z=1/\sqrt{2}$
		
* **type III (one-half pad loss):**
$\alpha=g^{+1/4}$

cut frequency is located at $G/2$

In [ ]:
fc = 1000  # Hz
G = -12  # dB
ShvType = "III"
Qz = 1/np.sqrt(2)  # Butterworth quality
Qp = Qz
B, A, b, a = af.biquad_hshv2nd(fc, G, Qz, Qp, fs, ShvType)
af.bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

## Exercise

1.) Check the differences of the types I,II vs. III for the PEQ and shelving filters w.r.t. their cut frequencies.

Hints: 

* The **type III** filters as for PEQs and shelving filters consistently define the cut frequency as the point of half gain in dB.

* Can you define the half gain $G/2$ cut frequencies of type III for small gain/attenuation $|G| <3$ dB?

* Can you define the '3 dB cut frequencies' for small gain/attenuation $|G| <3$ dB for type I and II?

* Where are the '3dB cut frequencies' located for type I,II for large gain or large attenuation? 

* Check how the filters of your favorite audio software are implemented.

2.) Discuss the differences of the **sin**, **cos**, **tan**-style quality pre-warping. For very high frequencies and rather small bandwidths the differences become obvious. 

In [ ]:
# high shelve example
Qz = 1/np.sqrt(2)
Qp = Qz
fc = 1000  # Hz
G = 15  # dB

# fc at 3 dB
B, A, b, a = af.biquad_hshv2nd(fc, G, Qz, Qp, fs, "II")
af.bode_plot(None, B, A, b, a, fs)

# fc at 7.5 dB
B, A, b, a = af.biquad_hshv2nd(fc, G, Qz, Qp, fs, "III")
af.bode_plot(None, B, A, b, a, fs)

# fc at 12 dB (15 dB - 3 dB)
B, A, b, a = af.biquad_hshv2nd(fc, G, Qz, Qp, fs, "I")
af.bode_plot(None, B, A, b, a, fs)

In [ ]:
# PEQ example
BW = 2  # bandwidth in octaves
fm = 2000  # mid frequency in Hz; using fm=1000 and BW=2, then fc is at 1 kHz
G = 15  # dB
QBP = af.q_from_bw(BW)
QWarpType = "cos"

B, A, b, a = af.biquad_peq2nd(fm, G, QBP, fs, "II", QWarpType)
af.bode_plot(None, B, A, b, a, fs)

B, A, b, a = af.biquad_peq2nd(fm, G, QBP, fs, "III", QWarpType)
af.bode_plot(None, B, A, b, a, fs)

# note: this type is not symmetrical w.r.t. 
#the same dB amount of gain or attenuation
B, A, b, a = af.biquad_peq2nd(fm, G, QBP, fs, "I", QWarpType)
af.bode_plot(None, B, A, b, a, fs)

# Excursus: Higher Order Filters

In audio signal processing also IIR filters of higher order are applied. However, they will be usually split into second order sections for series connection. Note that the group delay of an IIR filter increases with higher filter order. 

A prominent example of using higher order IIR filters is found in loudspeaker design. For a multi-way loudspeaker each driver must only be driven within an appropriate intended frequency range. This is usually realized with a bandpass filter that is built from a higher order lowpass and a higher order highpass filter. Thus, each loudspeaker exhibits its own bandpass filter. The acoustic summation of all driver's generated sound pressure then yields the desired full audio bandwidth.

Consider, the following two-driver application, in which the same bandpass characteristic is used for the low and high frequency driver resulting up in a 4th order so called Linkwitz-Riley frequency crossover.

In [ ]:
N = 2**14  # frequency resolution

# two equal 2nd order Butterworth filters in series
# is known as a 4th order Linkwitz-Riley filter
bi1 = 1 
ai1 = np.sqrt(2)
bi2 = 1 
ai2 = np.sqrt(2)

# low frequency driver range
fLow_LP = 1000
fLow_HP = 20

# high frequency driver range
fHigh_LP = 15000
fHigh_HP = fLow_LP

# note that Laplace domain filters are actually used in the code below,
# since z-domain is always overwritten 
# get transfer functions of LOW frequency driver's bandpass
# highpass
B, A, b, a = af.biquad_hp2nd(fLow_HP, fs, bi1, ai1)
W, LowHP1 = sig.freqz(b, a, N)
s, LowHP1 = sig.freqs(B, A, fs*W)
B, A, b, a = af.biquad_hp2nd(fLow_HP, fs, bi2, ai2)
W, LowHP2 = sig.freqz(b, a, N)
s, LowHP2 = sig.freqs(B, A, fs*W)
# lowpass
B, A, b, a = af.biquad_lp2nd(fLow_LP, fs, bi1, ai1)
W, LowLP1 = sig.freqz(b, a, N)
s, LowLP1 = sig.freqs(B, A, fs*W)
B, A, b, a = af.biquad_lp2nd(fLow_LP, fs, bi2, ai2)
W, LowLP2 = sig.freqz(b, a, N)
s, LowLP2 = sig.freqs(B, A, fs*W)

# get transfer function of HIGH frequency driver's bandpass
# highpass
B, A, b, a = af.biquad_hp2nd(fHigh_HP, fs, bi1, ai1)
W, HighHP1 = sig.freqz(b, a, N)
s, HighHP1 = sig.freqs(B, A, fs*W)
B, A, b, a = af.biquad_hp2nd(fHigh_HP, fs, bi2, ai2)
W, HighHP2 = sig.freqz(b, a, N)
s, HighHP2 = sig.freqs(B, A, fs*W)
# lowpass
B, A, b, a = af.biquad_lp2nd(fHigh_LP, fs, bi1, ai1)
W, HighLP1 = sig.freqz(b, a, N)
s, HighLP1 = sig.freqs(B, A, fs*W)
B, A, b, a = af.biquad_lp2nd(fHigh_LP, fs, bi2, ai2)
W, HighLP2 = sig.freqz(b, a, N)
s, HighLP2 = sig.freqs(B, A, fs*W)

f = fs*W / (2.*np.pi)
# series connection of four 2nd order filters:
Low = (LowHP1*LowHP2) * (LowLP1*LowLP2)  
High = (HighHP1*HighHP2) * (HighLP1*HighLP2)

Low[0] = 1e-15  # avoid zero at DC
High[0] = 1e-15  # avoid zero at DC

# simulation of the acoustic summation on both driver's middle axis:
AcousticSum = Low + High  

#plot with pyplot-interface
plt.figure(figsize = (16,9))
#plt.xkcd()  # comic style
plt.semilogx(f, 20*np.log10(np.abs(Low)), 'C0',
             label = r'electric bandpass for low frequency driver')
plt.semilogx(f, 20*np.log10(np.abs(High)), 'C1', 
             label = r'electric bandpass for high frequency driver')
plt.semilogx(f, 20*np.log10(np.abs(AcousticSum)), 'C2', 
             label = r'acoustic on-axis summation')
plt.autoscale("tight")
plt.title(r'4th-order Linkwitz-Riley X-Over for a Two-Way Loudspeaker')
plt.xlabel(r'$f$ / Hz')
plt.ylabel(r'20 log10 |$H$| / dB')
plt.axis([10, 20000, -30, +3])
plt.yticks(np.arange(-30, 3+3, 3));
plt.xticks((10, 20, 50, 100, 200, 500, 
            1000, 2000, 5000, 10000, 20000), 
           ["10", "20", "50", "100", "200", "500", 
            "1k", "2k", "5k", "10k", "20k"])
plt.xlim(10, 20000)
plt.ylim(-30, 3)
plt.grid(True, which="both", axis="both", 
         linestyle="-", linewidth=0.5, color=(0.8, 0.8, 0.8)) 
plt.legend(loc="best");
plt.show()

## Exercise

What filter characteristics has the acoustic sum? To explore it in detail, plot and discuss the phase of the Linwitz-Riley filters as well as of the acoustic sum.

Why sum the magnitudes of the low and the high frequency driver precisely to 0 dB at $f_c$ for the chosen example? Which phase shift exhibits the sum transfer function?

Is this handling also possible with Butterworth filters?

# Appendix

## Zoelzer PEQ (Type II, no Q prewarp)
We check type of \cite[Table 2.4]{Zoelzer2011}

In [ ]:
fm = 1000
Q = 2/3
G = 12   
B, A, b, a = af.biquad_peq2nd(fm, G, Q, fs, "II", "NoQPreWarp")

# U. Zoelzer (2011): "DAFX - Digital Audio Effects", 2nd, Wiley, Table 2.4:
bZ,aZ = af.biquad_peq2nd_zoelzer(fm, G, Q, fs)

np.testing.assert_allclose(bZ, b)
np.testing.assert_allclose(aZ, a)

## RBJ PEQ (Type III, Q prewarp: sin)
We check type \cite(Bristow-Johnson1994), eq. (16)

In [ ]:
fm = 1000
Q = 2/3
G = 12  
B, A, b, a = af.biquad_peq2nd(fm, G, Q, fs, "III", "sin")

# Robert Bristow-Johnson (1994): "The equivalence of various methods of
# computing biquad coefficients for audio parametric equalizers."
# In: Proc. of 97th AES Convention, San Fransisco, eq. (16)
bR, aR = af.biquad_peq2nd_RBJ(fm, G, Q, fs)

np.testing.assert_allclose(bR, b)
np.testing.assert_allclose(aR, a)

## Zoelzer 2nd Order Low-Shelving (Type I, Qz=Qp=1/sqrt(2))
We check type of \cite[Table 2.3]{Zoelzer2011}

In [ ]:
fc = 1000
G = 12
Qz = 1/np.sqrt(2)
Qp = 1/np.sqrt(2)    
B, A, b, a = af.biquad_lshv2nd(fc, G, Qz, Qp, fs, "I")    

# U. Zoelzer (2011): "DAFX - Digital Audio Effects", 2nd, Wiley, Table 2.3:
[bZ,aZ] = af.biquad_lshv2nd_Zoelzer(fc,G,fs)   

np.testing.assert_allclose(bZ, b)
np.testing.assert_allclose(aZ, a)

## RBJ 2nd Order Low-Shelving (Type III, Qz=Qp=1/sqrt(2) -> S = 1)
We check type of Robert Bristow-Johnson Audio EQ Cookbook

In [ ]:
fc = 1000
G = 12
Qz = 1/np.sqrt(2)
Qp = 1/np.sqrt(2)
S = 1
B, A, b, a = af.biquad_lshv2nd(fc, G, Qz, Qp, fs, "III")   

# http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt
bR, aR = af.biquad_lshv2nd_RBJ(fc, G, S, fs) 

np.testing.assert_allclose(bR, b)
np.testing.assert_allclose(aR, a)

## Zoelzer 2nd Order High-Shelving (Type I, Qz=Qp=1/sqrt(2))
We check type of \cite[Table 2.3]{Zoelzer2011}

In [ ]:
fc = 1000
G = 12
Qz = 1/np.sqrt(2)
Qp = 1/np.sqrt(2)    
B, A, b, a = af.biquad_hshv2nd(fc, G, Qz, Qp, fs, "I")    

# U. Zoelzer (2011): "DAFX - Digital Audio Effects", 2nd, Wiley, Table 2.3:
[bZ,aZ] = af.biquad_hshv2nd_Zoelzer(fc,G,fs)   

np.testing.assert_allclose(bZ, b)
np.testing.assert_allclose(aZ, a)

## RBJ 2nd Order High-Shelving (Type III, Qz=Qp=1/sqrt(2) -> S = 1)
We check type of Robert Bristow-Johnson Audio EQ Cookbook

In [ ]:
fc = 1000
G = 12
Qz = 1/np.sqrt(2)
Qp = 1/np.sqrt(2)
S = 1
B, A, b, a = af.biquad_hshv2nd(fc, G, Qz, Qp, fs, "III")   

# http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt
bR, aR = af.biquad_hshv2nd_RBJ(fc, G, S, fs) 

np.testing.assert_allclose(bR, b)
np.testing.assert_allclose(aR, a)

## Butterworth Filter Characteristics

Higher order filter design can be realized with series connection of biquads. The coefficients for **Butterworth** type are e.g. given as, cf. \cite{Tietze1978}

$\text{a}_\text{i}$ $\text{b}_\text{i}$     

**1st order**

1.0000 0.0000 

**2nd order**

1.4142 1.0000 (used in the example above)

**3rd order** (2 biquads in series connection with different bi, ai)

1.0000 0.0000

1.0000 1.0000

**4th order** (2 biquads in series connection with different bi, ai)

1.8478 1.0000 

0.7654 1.0000

# References

**Copyright**

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Sascha Spors, Digital Signal Processing - Lecture notes featuring computational examples, 2016-2018*.